In [1]:
library(pacman)
p_load(data.table, dplyr, ggplot2, viridis, magrittr, VennDiagram, ggpubr, limma, edgeR, tidyr, GenomicRanges, RColorBrewer, pheatmap, Seurat, fgsea, GSEABase)

cell_list <- c("N", "CM", "EM", "EMRA")
hd_list <- paste0("HD", c(1:3, 5:7))
expr_list = c("Input", "Product", "Stim1", "Stim2", "Stim3")
cell_comp_list = c("N_CM", "N_EM", "N_EMRA", "CM_EM", "CM_EMRA", "EM_EMRA")
hist_list <- c("H3K27me3", "H3K4me2")

in_path <- "CART_CUTRUN_Project/results/RNAseq/process/RSEM/"
out_path <- "CART_CUTRUN_Project/results/RNAseq/analysis/RSEM/"
fig_path <- "CART_CUTRUN_Project/results/paper_figure_pools/"


In [ ]:
## Master peak annotation
library(ChIPseeker)
load(file = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/RData/masterPeak_peakAnno_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData")
pdf("CART_CUTRUN_Project/results/paper_figure/SuppFig-QC/master_peak_annotation.pdf")
plotAnnoPie(mPeakAnno[["H3K27me3"]], title = "H3K27me3")
plotAnnoPie(mPeakAnno[["H3K4me2"]], title = "H3K4me2")
plotAnnoBar(mPeakAnno[["H3K27me3"]], title = "H3K27me3")
plotAnnoBar(mPeakAnno[["H3K4me2"]], title = "H3K4me2")
plotDistToTSS(mPeakAnno[["H3K27me3"]], title = "H3K27me3")
plotDistToTSS(mPeakAnno[["H3K4me2"]], title = "H3K4me2")
dev.off()

library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(org.Hs.eg.db)
options(ChIPseeker.ignore_1st_exon = TRUE)
options(ChIPseeker.ignore_1st_intron = TRUE)
options(ChIPseeker.ignore_downstream = TRUE)
options(ChIPseeker.ignore_promoter_subcategory = TRUE)

txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene
annodb <- "org.Hs.eg.db"
for(hist in c("H3K27me3", "H3K4me2")){
    for(ct in c("N", "CM", "EM", "EMRA")){
        peak_gr <- GRanges()
        for(hd in paste0("HD", 4:7)){
            name = paste(hist, ct, "Input", hd, sep = "_")
            peak_gr = append(peak_gr, peakAll[[name]])
        }
        peak_ct_anno = annotatePeak(reduce(peak_gr), tssRegion = c(-2000, 0), addFlankGeneInfo = TRUE,  TxDb = txdb, annoDb = annodb,  level = "gene")
        pdf(paste0("CART_CUTRUN_Project/results/paper_figure/SuppFig-QC/", hist, "_", ct, "_annotation.pdf"))
        print(plotAnnoPie(peak_ct_anno, title = paste0(hist, "_", ct)))
        print(plotAnnoBar(peak_ct_anno, title = paste0(hist, "_", ct)))
        print(plotDistToTSS(peak_ct_anno, title = paste0(hist, "_", ct)))
        dev.off()
   }
}

In [ ]:
## -	Numbers of differentially expressed genes (RNAseq) in transcription of input of N vs CM vs EM vs EMRA - Venn diagram

## 1. Pairwise comparisons
## VOOM normalized and use donor id as random effect to remove donor variation
## save(data, selectR, dataS, voomDDS, results, file = paste0(out_path, "/RData/HD1-3_5-7_normLimma_perExprCondition.RData"))
load(file = paste0(out_path, "/RData/HD1-3_5-7_normLimma_perExprCondition.RData"))

de_num <- c()
for(cell_comp in cell_comp_list){
    de_num <- data.frame(cond = "Input", comparison = cell_comp, de_num = results[[paste0("Input_", cell_comp)]] %>% filter(Significance != "notDE") %>% nrow) %>%
    rbind(de_num)
}
de_num$comparison <- factor(de_num$comparison, levels = cell_comp_list)

de_num %>% ggplot(aes(x = comparison, y = de_num, label = de_num, fill = comparison)) +
geom_bar(stat = "identity") +
geom_text(vjust = -0.1) +
theme_bw(base_size = 20) +
xlab("") +
ylab("# of Differentially Expressed Genes") +
ggtitle("Input (FDR <= 0.05 |logFC| >= 1)") +
scale_fill_viridis(discrete = TRUE) +
rotate_x_text(angle = 90) +
rremove("legend")
ggsave(filename = paste0(fig_path, "RNAseq_pairwise_de_bar_plot.pdf"), width = 8, height = 7)


In [ ]:

## 2. N vs rest, CM vs rest, ... for venn diagram
get_contrast <- function(cell_type, design){

  if(cell_type == "N"){
    contrast <- makeContrasts(
      Input_N_Others = Others.Input - N.Input,
      Product_N_Others = Others.Product - N.Product,
      Stim1_N_Others = Others.Stim1 - N.Stim1,
      Stim2_N_Others = Others.Stim2 - N.Stim2,
      Stim3_N_Others = Others.Stim3 - N.Stim3,
      levels = design
    )
  }
  if(cell_type == "CM"){
    contrast <- makeContrasts(
      Input_CM_Others = Others.Input - CM.Input,
      Product_CM_Others = Others.Product - CM.Product,
      Stim1_CM_Others = Others.Stim1 - CM.Stim1,
      Stim2_CM_Others = Others.Stim2 - CM.Stim2,
      Stim3_CM_Others = Others.Stim3 - CM.Stim3,
      levels = design
    )
  }
  if(cell_type == "EM"){
    contrast <- makeContrasts(
      Input_EM_Others = Others.Input - EM.Input,
      Product_EM_Others = Others.Product - EM.Product,
      Stim1_EM_Others = Others.Stim1 - EM.Stim1,
      Stim2_EM_Others = Others.Stim2 - EM.Stim2,
      Stim3_EM_Others = Others.Stim3 - EM.Stim3,
      levels = design
    )
  }
  if(cell_type == "EMRA"){
    contrast <- makeContrasts(
      Input_EMRA_Others = Others.Input - EMRA.Input,
      Product_EMRA_Others = Others.Product - EMRA.Product,
      Stim1_EMRA_Others = Others.Stim1 - EMRA.Stim1,
      Stim2_EMRA_Others = Others.Stim2 - EMRA.Stim2,
      Stim3_EMRA_Others = Others.Stim3 - EMRA.Stim3,
      levels = design
    )
  }
  return(contrast)
}

data <- c()
col_name_list <- c()
for (cell in cellList) {
  for (expr in exprList) {
    for (hd in hdList) {
      ## print(paste(expr, cell, hd, sep = "_"))
      dataTmp <- fread(paste0(in_path, "RNA_CD8_", cell, "_", expr, "_", hd, ".genes.results")) %>% dplyr::select(expected_count)
      data <- cbind(data, round(dataTmp))
      col_name_list <- c(col_name_list, paste0(expr, "_", cell, "_", hd))
    }
  }
}
geneID <- fread(paste0(in_path, "RNA_CD8_N_Input_HD1.genes.results"))$gene_id
rownames(data) <- geneID
colnames(data) <- col_name_list ## paste(rep(rep(expr_list, each = length(hd_list)), length(cell_list)), rep(cell_list, each = length(hd_list)*length(expr_list)), rep(hd_list, length(expr_list)*length(cell_list)), sep = "_")

## Filter and delete low expressed genes
selectR <- which(rowSums(data) > 10) ## remove low count genes
dataS <- data[selectR, ]
geneNameList <- rownames(data)[selectR]


for (target_cell in c("CM", "EM", "EMRA")) { ## "N"
  target <- data.frame(humanDonor = rep(hdList, length(exprList) * length(cellList)), cell = rep(cellList, each = length(hdList) * length(exprList)), expr = rep(rep(exprList, each = length(hdList))))
  target$cell[which(target$cell != target_cell)] <- "Others"


  ## Experimental design
  treat <- factor(paste(target$cell, target$expr, sep = "."))
  design <- model.matrix(~ 0 + treat)
  colnames(design) <- levels(treat)
  design %>% head()
  contrast <- get_contrast(target_cell, design)
  contrast %>% head()

  
  voomDDS <- voom(counts = dataS, design = design, normalize.method = "cyclicloess", plot = TRUE)
  ## option 1 using voomDDS option2 using normDDS as normalized input.
  inputDDS <- voomDDS
  corfit <- duplicateCorrelation(inputDDS, design, block = target$humanDonor)
  # corfit$consensus
  fit <- lmFit(inputDDS, design, block = target$humanDonor, correlation = corfit$consensus)
  fitContrast <- contrasts.fit(fit, contrast)
  fitBayes <- eBayes(fitContrast, robust = TRUE)

  results <- list()
  for (i in 1:ncol(contrast)) {
    #- Results
    res <- topTable(fit = fitBayes, adjust.method = "fdr", coef = i, number = nrow(inputDDS), sort = "P")

    res <- data.table(GeneName = geneNameList[as.numeric(rownames(res))], GeneIndex = rownames(res) %>% as.numeric(), res)
    res[, Significance := ifelse((adj.P.Val <= 0.05 & sign(logFC) == 1 & abs(logFC) >= 1), "Up",
      ifelse((adj.P.Val <= 0.05 & sign(logFC) == -1 & abs(logFC) >= 1), "Down", "notDE")
    )]
    results[[i]] <- res
    #- Output
    write.table(res, file = paste0(out_path, "/limma_tables_HD1-3_5-7/one_vs_rest/DE_", colnames(contrast)[i], "_adj0.05_logFC1.csv"), quote = FALSE, row.names = FALSE, sep = ",")
  }
  names(results) <- colnames(contrast)
  save(data, selectR, dataS, voomDDS, results, file = paste0(out_path, "/RData/HD1-3_5-7_normLimma_perExprCondition_one_vs_rest_", target_cell, ".RData"))
}

de_list <- list()
for(target_cell in c("N", "CM", "EM", "EMRA")){
    load(file = paste0(out_path, "/RData/HD1-3_5-7_normLimma_perExprCondition_one_vs_rest_", target_cell, ".RData"))
    de_list[[target_cell]] <- results[[paste0("Input_", target_cell, "_Others")]] %>% filter(Significance != "notDE") %$% GeneName
}
# results
pdf(paste0(fig_path, "RNAseq_DE_Input_Venn_diagram.pdf"))
venn.diagram(
    x = list(de_list[["N"]], de_list[["CM"]], de_list[["EM"]], de_list[["EMRA"]]),
    category.names = c(
        paste0("N (", length(de_list[["N"]]), ")"), 
        paste0("CM (", length(de_list[["CM"]]), ")"),
        paste0("EM (", length(de_list[["EM"]]), ")"),
        paste0("EMRA (", length(de_list[["EMRA"]]), ")")),
    filename = NULL, #paste0(fig_path, "RNAseq_DE_Input_Venn_diagram.pdf"
    col = c("#440154", "#31688e", "#35b779", "#fde725"), 
    fill = c(alpha("#440154", 0.3), alpha("#31688e", 0.3), alpha("#35b779", 0.3), alpha("#fde725", 0.3)),
    resolution = 1000, 
    disable.logging = TRUE
) %>% grid.draw


dev.off()

In [ ]:
## -	We used CUT&RUN to mark H3K27me3 and H3K4me2 regions. We used an isotype control to control for non-specifically bound DNA
## 1. diagram: refer to cut&tag tutorial diagram
## 2. QC outputs

## 2.1 Sequencing depth
seqDepTmp = fread("CART_CUTRUN_Project/results/CUTANDRUN/analysis/sequenceDepth.csv", header = FALSE)
seqDep = separate(seqDepTmp, "V1", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") %>% mutate(CellType = factor(toupper(CellType), levels = c("BULK", "N", "CM", "EM", "EMRA")))

## data = seqDep %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", CellType != "BULK", Experiment != "PatientIP") %>% filter(!(Experiment == "Input" & (HD == "HD1" | HD == "HD2" | HD == "HD3")))  #, Experiment != "PatientIP"
data = seqDep %>% filter(Experiment == "Input", CD == "CD8", CellType != "MAIT", CellType != "NA", CellType != "BULK", Experiment != "PatientIP") %>% filter(!(Experiment == "Input" & (HD == "HD1" | HD == "HD2" | HD == "HD3")))  #, Experiment != "PatientIP"

data %>% ggplot(aes(x = CellType, y = as.numeric(V2)/1000000, fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    ## geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) + 
    ## geom_jitter(position = position_jitter(0.15)) + 
    # geom_label_repel(data = data %>% filter(Experiment == "Input", HD %in% paste0("HD", c(1, 2, 3, 6))), aes(label = paste0(Experiment, "_", HD)), position = position_jitterdodge()) +
    facet_grid(~Histone, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("Sequencing Depth per Million") +
    xlab("") 
ggsave(filename = paste0(fig_path, "cutrun_seqdepth_before_alignment.pdf"), width = 10, height = 7)

## 2.2 Alignment Rate
alignRateTmp = fread("CART_CUTRUN_Project/results/CUTANDRUN/analysis/alignmentRate.csv", header = FALSE)
alignRate = separate(alignRateTmp, "V1", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") %>% mutate(CellType = factor(toupper(CellType), levels = c("BULK", "N", "CM", "EM", "EMRA")))

## dataAR = alignRate %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA")  %>% filter(!(Experiment == "Input" & (HD == "HD1" | HD == "HD2" | HD == "HD3"))) #, Experiment != "PatientIP") 
dataAR = alignRate %>% filter(Experiment == "Input", CD == "CD8", CellType != "MAIT", CellType != "NA")  %>% filter(!(Experiment == "Input" & (HD == "HD1" | HD == "HD2" | HD == "HD3"))) #, Experiment != "PatientIP") 

dataM = left_join(data, dataAR, by = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate")) %>% mutate(mappN = V2.x * V2.y/100)

dataM %>% ggplot(aes(x = CellType, y = as.numeric(mappN)/1000000, fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    ## geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) + 
    ## geom_jitter(position = position_jitter(0.15)) + 
    # geom_label_repel(data = data %>% filter(Experiment == "Input", HD %in% paste0("HD", c(1, 2, 3, 6))), aes(label = paste0(Experiment, "_", HD)), position = position_jitterdodge()) +
    facet_grid(~Histone, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("# of Mappable Read-pairs per Million") +
    xlab("")
ggsave(filename = paste0(fig_path, "cutrun_seqdepth_after_alignment.pdf"), width = 10, height = 7)

dataHR = alignRate %>% filter(Experiment == "Input", CD == "CD8", CellType != "MAIT", CellType != "NA", CellType != "BULK", Experiment != "PatientIP") %>% filter(!(Experiment == "Input" & (HD == "HD1" | HD == "HD2" | HD == "HD3"))) 

dataHR %>% ggplot(aes(x = CellType, y = as.numeric(V2), fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    ## geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    geom_jitter(position = position_jitter(0.15)) +
    facet_grid(~Histone, scales = "free") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("Alignment Rate") +
    xlab("") 
ggsave(filename = paste0(fig_path, "cutrun_alignment_rate.pdf"), width = 10, height = 7)

## 2.3 Duplicate rate
dupRateTmp = fread("CART_CUTRUN_Project/results/CUTANDRUN/analysis/validDupRmRate.csv", header = FALSE)
dupRate = separate(dupRateTmp, "V1", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") %>% mutate(DupRate = 100 - V3/V2*100, CellType = factor(toupper(CellType), levels = c("BULK", "N", "CM", "EM", "EMRA")))

data = dupRate %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", CellType != "BULK", Experiment != "PatientIP") %>% filter(!(Experiment == "Input" & (HD == "HD1" | HD == "HD2" | HD == "HD3"))) 

data %>% ggplot(aes(x = CellType, y = as.numeric(DupRate), fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    #geom_label_repel(data = data %>% filter(Experiment == "Input", HD %in% paste0("HD", c(1, 2, 3, 6))), aes(label = paste0(Experiment, "_", HD)), position = position_jitterdodge()) +
    facet_grid(~Histone, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("Duplicates Rate") +
    xlab("") +
    ylim(0, 100)
ggsave(filename = paste0(fig_path, "cutrun_duplicates_rate.pdf"), width = 10, height = 7)

data = dupRate %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", CellType != "BULK", Experiment != "PatientIP") %>% filter(!(Experiment == "Input" & (HD == "HD1" | HD == "HD2" | HD == "HD3"))) 

data %>% ggplot(aes(x = CellType, y = as.numeric(V3)/1000000, fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    # geom_label_repel(data = data %>% filter(Experiment == "Input", HD %in% paste0("HD", c(1, 2, 3, 6))), aes(label = paste0(Experiment, "_", HD)), position = position_jitterdodge()) +
    facet_grid(~Histone, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("Estimated Unique Library Size per Million") +
    xlab("") 
ggsave(filename = paste0(fig_path, "cutrun_unique_library_size.pdf"), width = 10, height = 7)

## 2.4 Fragment length
inPath = "CART_CUTRUN_Project/results/CUTANDRUN/process/"
fileList = fread("CART_CUTRUN_Project/meta/Histone_IgG_pairing.csv")$histone
fragL = c()
for(file in fileList){
    fragL = fread(paste0(inPath, file, "/alignment/bowtie2_align.fragLen.freq")) %>% mutate(info = file, Weight = V2/sum(V2)) %>% rbind(fragL, .)

}

fileList = fread("CART_CUTRUN_Project/meta/Histone_IgG_pairing.csv")$IgG
for(file in fileList){
    fragL = fread(paste0(inPath, file, "/alignment/bowtie2_align.fragLen.freq")) %>% mutate(info = file, Weight = V2/sum(V2)) %>% rbind(fragL, .)

}
fragLenD = separate(fragL, "info", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") %>% mutate(CellType = factor(toupper(CellType), levels = c("BULK", "N", "CM", "EM", "EMRA"))) %>% rename(fragLen = V1, Freq = V2)

fragData = fragLenD %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", CellType != "BULK", Experiment != "PatientIP") %>% filter(!(Experiment == "Input" & (HD == "HD1" | HD == "HD2" | HD == "HD3"))) 

fragData %>% ggplot(aes(x = CellType, y = as.numeric(fragLen), weight = Weight, fill = CellType)) +
    geom_violin(bw = 1) +
    # geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    facet_grid(~Histone, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("Fragment Length") +
    xlab("") +
    scale_y_continuous(breaks = seq(50, 700, 50)) +
    rremove("legend")

ggsave(filename = paste0(fig_path, "cutrun_fragment_length.pdf"), width = 10, height = 7)

In [30]:
## 3 Peak calling
inPath = "CART_CUTRUN_Project/results/CUTANDRUN/process/"
outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"
fileList = fread("CART_CUTRUN_Project/meta/Histone_IgG_pairing.csv")$histone

convertGR = function(peak){
    if(nrow(peak) > 0){
        peak.gr = GRanges(seqname = peak$V1, IRanges(peak$V2, peak$V3), strand = "*")
    }else{
        peak.gr = GRanges()
    }
    return(peak.gr)
}
# peakN = c()
# peakLen = c()
# peakCov = c()
# peakOverlap = c()
# for(file in fileList){
#     print(file)
#     macsNarrow = fread(paste0(inPath, file, "/peakCalling/normLibSize/MACS2/macs2_narrow_q_0.05_peaks.narrowPeak")) %>% filter(V1 != "chrY", V1 != "chrM")
#     macsBroad = fread(paste0(inPath, file, "/peakCalling/normLibSize/MACS2/macs2_broad_q_0.05_peaks.broadPeak")) %>% filter(V1 != "chrY", V1 != "chrM")
#     seacrControl = fread(paste0(inPath, file, "/peakCalling/normLibSize/SEACR/seacr_control_seacrNorm.peaks.stringent.bed")) %>% filter(V1 != "chrY", V1 != "chrM")
#     seacrTop = fread(paste0(inPath, file, "/peakCalling/normLibSize/SEACR/seacr_top0.1_spikeInNorm.peaks.stringent.bed")) %>% filter(V1 != "chrY", V1 != "chrM")
    
    
#     peakN = data.frame(peakN = c(macsNarrow %>% nrow, macsBroad %>% nrow, seacrControl %>% nrow, seacrTop %>% nrow), type = c("MACS2_Narrow", "MACS2_Broad", "SEACR_Control", "SEACR_Top10%"), info = file) %>% rbind(peakN, .)
    
#     peakLen = data.frame(peakLen = c(macsNarrow$V3 - macsNarrow$V2, macsBroad$V3 - macsBroad$V2, seacrControl$V3 - seacrControl$V2, seacrTop$V3 - seacrTop$V2), type = c(rep("MACS2_Narrow", nrow(macsNarrow)), rep("MACS2_Broad", nrow(macsBroad)), rep("SEACR_Control", nrow(seacrControl)), rep("SEACR_Top10%", nrow(seacrTop))), info = file) %>% rbind(peakLen, .)
    
#     peakCov = data.frame(peakLen = c(sum(macsNarrow$V3 - macsNarrow$V2), sum(macsBroad$V3 - macsBroad$V2),  sum(seacrControl$V3 - seacrControl$V2), sum(seacrTop$V3 - seacrTop$V2)), type = c("MACS2_Narrow", "MACS2_Broad", "SEACR_Control", "SEACR_Top10%"), info = file) %>% rbind(peakCov, .)
    
#     macsNarrow.gr = convertGR(macsNarrow)
#     macsBroad.gr = convertGR(macsBroad)
#     seacrControl.gr = convertGR(seacrControl)
#     seacrTop.gr = convertGR(seacrTop)
    
#     narrowControl = findOverlaps(macsNarrow.gr, seacrControl.gr)
#     broadControl = findOverlaps(macsBroad.gr, seacrControl.gr)
#     narrowTop = findOverlaps(macsNarrow.gr, seacrTop.gr)
#     broadTop = findOverlaps(macsBroad.gr, seacrTop.gr)

#     peakOverlap = data.frame(
#         overlapN = c(
#             narrowControl@from %>% unique %>% length, 
#             narrowControl@to %>% unique %>% length,
#             broadControl@from %>% unique %>% length, 
#             broadControl@to %>% unique %>% length,
#             narrowTop@from %>% unique %>% length,
#             narrowTop@to %>% unique %>% length,
#             broadTop@from %>% unique %>% length, 
#             broadTop@to %>% unique %>% length
#         ),
#         overlapProp = c(
#             (narrowControl@from %>% unique %>% length)/length(macsNarrow.gr) * 100, 
#             (narrowControl@to %>% unique %>% length)/length(seacrControl.gr) * 100,
#             (broadControl@from %>% unique %>% length)/length(macsBroad.gr) * 100, 
#             (broadControl@to %>% unique %>% length)/length(seacrControl.gr) * 100,
#             (narrowTop@from %>% unique %>% length)/length(macsNarrow.gr) * 100,
#             (narrowTop@to %>% unique %>% length)/length(seacrTop.gr) * 100,
#             (broadTop@from %>% unique %>% length)/length(macsBroad.gr) * 100, 
#             (broadTop@to %>% unique %>% length)/length(seacrTop.gr) * 100
#         ),
#         propLabel = c(
#             "MACS2_Narrow vs SEACR_Control/MACS2_Narrow", 
#             "MACS2_Narrow vs SEACR_Control/SEACR_Control", 
#             "MACS2_Broad vs SEACR_Control/MACS2_Broad", 
#             "MACS2_Broad vs SEACR_Control/SEACR_Control", 
#             "MACS2_Narrow vs SEACR_Top/MACS2_Narrow", 
#             "MACS2_Narrow vs SEACR_Top/SEACR_Top", 
#             "MACS2_Broad vs SEACR_Top/MACS2_Broad", 
#             "MACS2_Broad vs SEACR_Top/SEACR_Top"
#         ), 
#         numLabel = c(
#             "MACS2_Narrow vs SEACR_Control: MACS2_Narrow", 
#             "MACS2_Narrow vs SEACR_Control: SEACR_Control", 
#             "MACS2_Broad vs SEACR_Control: MACS2_Broad", 
#             "MACS2_Broad vs SEACR_Control: SEACR_Control", 
#             "MACS2_Narrow vs SEACR_Top: MACS2_Narrow", 
#             "MACS2_Narrow vs SEACR_Top: SEACR_Top", 
#             "MACS2_Broad vs SEACR_Top: MACS2_Broad", 
#             "MACS2_Broad vs SEACR_Top: SEACR_Top"
#         ), 
#         info = file
#     ) %>% rbind(peakOverlap, .)
    
# }
# save(peakN, peakCov, peakLen, peakOverlap, file = paste0(outPath, "RData/peakN_Cov_Len_summary_macs_narrow_broad_q_0.05_seacr_control_top10.RData")) ##  peakOverlap,
load(file = paste0(outPath, "RData/peakN_Cov_Len_summary_macs_narrow_broad_q_0.05_seacr_control_top10.RData"))

In [ ]:
# 3 Comparison of MACS2 and SEACR
# 3.1 Numbers of peaks
peakNData = separate(peakN, "info", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") %>% mutate(CellType = factor(toupper(CellType), levels = c("BULK", "N", "CM", "EM", "EMRA")))
peakNData$type[peakNData$type == "SEACR_Top1%"] <- "SEACR_Top10%"
peakNData$type <- factor(
    peakNData$type, 
    levels = c("MACS2_Narrow", "MACS2_Broad", "SEACR_Control", "SEACR_Top10%"), 
    labels = c("MACS2 Narrow", "MACS2 Broad", "SEACR Control", "SEACR Top10%")
)

peakNData %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", Experiment != "PatientIP") %>% ggplot(aes(x = CellType, y = as.numeric(peakN), fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    facet_grid(Histone~type, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("# of Peaks") +
    xlab("") 

ggsave(filename = paste0(fig_path, "cutrun_peak_number_macs_q_0.05_seacr_top10.pdf"), width = 12, height = 7)


In [ ]:

# 3.2	Peak width and coverage
peakLenData = separate(peakLen, "info", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") %>% 
mutate(CellType = factor(toupper(CellType), levels = c("N", "CM", "EM", "EMRA")))

save(peakLenData, file = paste0(outPath, "RData/peakOverlap_peakLenData_macs_q_0.05_seacr_top10.RData"))


# load(file = paste0(outPath, "RData/peakOverlap_peakLenData_macs_q_0.05_seacr_top10.RData"))
# dim(peakLenData)

peakLenData$type[peakLenData$type == "SEACR_Top1%"] <- "SEACR_Top10%"
peakLenData$type <- factor(
    peakLenData$type, 
    levels = c("MACS2_Narrow", "MACS2_Broad", "SEACR_Control", "SEACR_Top10%"), 
    labels = c("MACS2 Narrow", "MACS2 Broad", "SEACR Control", "SEACR Top10%")
)

peakLenData %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", Experiment != "PatientIP") %>% ggplot(aes(x = CellType, y = as.numeric(peakLen), fill = CellType)) +
    geom_violin() +
    # geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    facet_grid(Histone~type, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("Length of Peaks (bp)") +
    xlab("") 
ggsave(filename = paste0(fig_path, "cutrun_peak_length_macs_q_0.05_seacr_top10_violinplot.pdf"), width = 12, height = 7)

peakLenData %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", Experiment != "PatientIP") %>% ggplot(aes(x = CellType, y = as.numeric(peakLen), fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    facet_grid(Histone~type, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("Length of Peaks (bp)") +
    xlab("") +
    coord_cartesian(ylim = c(0, 15000)) 
ggsave(filename = paste0(fig_path, "cutrun_peak_length_macs_q_0.05_seacr_top10_boxplot.pdf"), width = 12, height = 7)

peakCovData = separate(peakCov, "info", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") %>% mutate(CellType = factor(toupper(CellType), levels = c("BULK", "N", "CM", "EM", "EMRA")))
peakCovData$type[peakCovData$type == "SEACR_Top1%"] <- "SEACR_Top10%"
peakCovData$type <- factor(
    peakCovData$type, 
    levels = c("MACS2_Narrow", "MACS2_Broad", "SEACR_Control", "SEACR_Top10%"), 
    labels = c("MACS2 Narrow", "MACS2 Broad", "SEACR Control", "SEACR Top10%")
)
peakCovData %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", Experiment != "PatientIP") %>% ggplot(aes(x = CellType, y = as.numeric(peakLen), fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    facet_grid(Histone~type, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("Coverage of Peaks (bp)") +
    xlab("") 

ggsave(filename = paste0(fig_path, "cutrun_peak_coverage_macs_q_0.05_seacr_top10.pdf"), width = 12, height = 7)


# # 3.3	Method comparison

peakOverlapData = separate(peakOverlap, "info", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") %>% mutate(CellType = factor(toupper(CellType), levels = c("BULK", "N", "CM", "EM", "EMRA")))
peakOverlapData$propLabel <- factor(
    peakOverlapData$propLabel, 
    levels = c("MACS2_Narrow vs SEACR_Control/MACS2_Narrow", 
            "MACS2_Narrow vs SEACR_Control/SEACR_Control", 
            "MACS2_Broad vs SEACR_Control/MACS2_Broad", 
            "MACS2_Broad vs SEACR_Control/SEACR_Control", 
            "MACS2_Narrow vs SEACR_Top/MACS2_Narrow", 
            "MACS2_Narrow vs SEACR_Top/SEACR_Top", 
            "MACS2_Broad vs SEACR_Top/MACS2_Broad", 
            "MACS2_Broad vs SEACR_Top/SEACR_Top"), 
    labels = c("Narrow vs Control/Narrow", 
            "Narrow vs Control/Control", 
            "Broad vs Control/Broad", 
            "Broad vs Control/Control", 
            "Narrow vs Top/Narrow", 
            "Narrow vs Top/Top", 
            "Broad vs Top/Broad", 
            "Broad vs Top/Top")
)
peakOverlapData %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", Experiment != "PatientIP") %>% ggplot(aes(x = CellType, y = as.numeric(overlapProp), fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    facet_grid(propLabel~Histone, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("% of Peaks Overlapped") +
    xlab("") 
ggsave(filename = paste0(fig_path, "cutrun_peak_overlap_prop_macs_q_0.05_seacr_top10.pdf"), width = 10, height = 25)


peakOverlapData %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", Experiment != "PatientIP") %>% ggplot(aes(x = CellType, y = as.numeric(overlapN), fill = CellType)) +
    geom_boxplot(outlier.shape = NA) +
    geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    facet_grid(numLabel~Histone, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("# of Peaks Overlapped") +
    xlab("") 
ggsave(filename = paste0(fig_path, "cutrun_peak_overlap_num_macs_q_0.05_seacr_top10.pdf"), width = 10, height = 25)


In [ ]:
# 3.4	Precision across donors
inPath = "CART_CUTRUN_Project/results/CUTANDRUN/process/"
outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"

blackList = c("H3K27me3_CD8_CM_Input_HD1_rep1", "H3K27me3_CD8_CM_Input_HD3_rep2", "H3K27me3_CD8_N_Input_HD2_rep1", "H3K27me3_CD8_N_Input_HD3_rep2")
fileList = fread("CART_CUTRUN_Project/meta/Histone_IgG_pairing.csv") %>% select(histone) %>% mutate(path = histone) %>% filter(! histone %in% blackList) %>% separate("histone", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") 
fileListUni = fileList %>% select(Histone, CD, CellType, Experiment) %>% unique
convertGR = function(peak){
    peak.gr = GRanges(seqname = peak$V1, IRanges(peak$V2, peak$V3), strand = "*")
    return(peak.gr)
}

reprodD = c()
for(i in 1:nrow(fileListUni)){
  print(i)
  hist = fileListUni[i, 1] %>% as.character
  cd = fileListUni[i, 2] %>% as.character
  cell = fileListUni[i, 3] %>% as.character
  exp = fileListUni[i, 4] %>% as.character
  hdList = fileList %>% filter(Histone == hist, CD == cd, CellType == cell, Experiment == exp)
  for(p1 in 1:nrow(hdList)){
      for(p2 in 1:nrow(hdList)){
        if(p1 != p2){
          
          ## MACS2 Narrow
          peak1.gr = fread(paste0(inPath, hdList$path[p1], "/peakCalling/normLibSize/MACS2/macs2_narrow_q_0.05_peaks.narrowPeak")) %>% filter(V1 != "chrY", V1 != "chrM") %>% convertGR()
          peak2.gr = fread(paste0(inPath, hdList$path[p2], "/peakCalling/normLibSize/MACS2/macs2_narrow_q_0.05_peaks.narrowPeak")) %>% filter(V1 != "chrY", V1 != "chrM") %>% convertGR()
          overlap = findOverlaps(peak1.gr, peak2.gr)
          reprodD = data.frame(info = hdList$path[p1], overlapHD = hdList$HD[p2], overlapN = overlap@from %>% unique %>% length, overlapP = overlap@from %>% unique %>% length/length(peak1.gr) * 100, peakN = length(peak1.gr), caller = "MACS2_Narrow") %>% rbind(reprodD)
          
          ## MACS2 Broad
          peak1.gr = fread(paste0(inPath, hdList$path[p1], "/peakCalling/normLibSize/MACS2/macs2_broad_q_0.05_peaks.broadPeak")) %>% filter(V1 != "chrY", V1 != "chrM") %>% convertGR()
          peak2.gr = fread(paste0(inPath, hdList$path[p2], "/peakCalling/normLibSize/MACS2/macs2_broad_q_0.05_peaks.broadPeak")) %>% filter(V1 != "chrY", V1 != "chrM") %>% convertGR()
          overlap = findOverlaps(peak1.gr, peak2.gr)
          reprodD = data.frame(info = hdList$path[p1], overlapHD = hdList$HD[p2], overlapN = overlap@from %>% unique %>% length, overlapP = overlap@from %>% unique %>% length/length(peak1.gr) * 100, peakN = length(peak1.gr), caller = "MACS2_Broad") %>% rbind(reprodD)
         
           ## SEACR Control
          peak1.gr = fread(paste0(inPath, hdList$path[p1], "/peakCalling/normLibSize/SEACR/seacr_control_seacrNorm.peaks.stringent.bed")) %>% filter(V1 != "chrY", V1 != "chrM") %>% convertGR()
          peak2.gr = fread(paste0(inPath, hdList$path[p2], "/peakCalling/normLibSize/SEACR/seacr_control_seacrNorm.peaks.stringent.bed")) %>% filter(V1 != "chrY", V1 != "chrM") %>% convertGR()
          overlap = findOverlaps(peak1.gr, peak2.gr)
          reprodD = data.frame(info = hdList$path[p1], overlapHD = hdList$HD[p2], overlapN = overlap@from %>% unique %>% length, overlapP = overlap@from %>% unique %>% length/length(peak1.gr) * 100, peakN = length(peak1.gr), caller = "SEACR_Control") %>% rbind(reprodD)
    
          p1N = length(peak1.gr)
          p2N = length(peak2.gr)
          peak1Con.gr = peak1.gr
          peak2Con.gr = peak2.gr
          
          ## SEACR Top
          peak1.gr = fread(paste0(inPath, hdList$path[p1], "/peakCalling/normLibSize/SEACR/seacr_top0.1_spikeInNorm.peaks.stringent.bed")) %>% filter(V1 != "chrY", V1 != "chrM") %>% convertGR()
          peak2.gr = fread(paste0(inPath, hdList$path[p2], "/peakCalling/normLibSize/SEACR/seacr_top0.1_spikeInNorm.peaks.stringent.bed")) %>% filter(V1 != "chrY", V1 != "chrM") %>% convertGR()
          overlap = findOverlaps(peak1.gr, peak2.gr)
          reprodD = data.frame(info = hdList$path[p1], overlapHD = hdList$HD[p2], overlapN = overlap@from %>% unique %>% length, overlapP = overlap@from %>% unique %>% length/length(peak1.gr) * 100, peakN = length(peak1.gr), caller = "SEACR_Top") %>% rbind(reprodD)
          
          # ## SEACR Control in Top
          # peak1ConTop.gr = peak1Con.gr[findOverlaps(peak1Con.gr, peak1.gr)@from %>% unique]
          # peak2ConTop.gr = peak2Con.gr[findOverlaps(peak2Con.gr, peak2.gr)@from %>% unique]
          # overlap = findOverlaps(peak1ConTop.gr, peak2ConTop.gr)
          # reprodD = data.frame(info = hdList$path[p1], overlapHD = hdList$HD[p2], overlapN = overlap@from %>% unique %>% length, overlapP = overlap@from %>% unique %>% length/length(peak1ConTop.gr) * 100, peakN = length(peak1ConTop.gr), caller = "SEACR_ControlInTop") %>% rbind(reprodD)
          
          # ## MACS2 - topN
          # peak1.gr = fread(paste0(inPath, hdList$path[p1], "/peakCalling/relax/MACS2/macs2_noControl_peaks.narrowPeak")) %>% filter(V1 != "chrY", V1 != "chrM") %>% top_n(p1N, wt = V9) %>% convertGR()
          # peak2.gr = fread(paste0(inPath, hdList$path[p2], "/peakCalling/relax/MACS2/macs2_noControl_peaks.narrowPeak")) %>% filter(V1 != "chrY", V1 != "chrM") %>% top_n(p2N, wt = V9) %>% convertGR()
          # overlap = findOverlaps(peak1.gr, peak2.gr)
          # reprodD = data.frame(info = hdList$path[p1], overlapHD = hdList$HD[p2], overlapN = overlap@from %>% unique %>% length, overlapP = overlap@from %>% unique %>% length/length(peak1.gr) * 100, peakN = length(peak1.gr), caller = "MACS2(TopN=SEACR_Control)") %>% rbind(reprodD)
         }
        
      }
    
  }
  
}
reprodD$caller = factor(reprodD$caller, levels = c("MACS2_Narrow", "MACS2_Broad", "SEACR_Control", "SEACR_Top")) ##"MACS2(TopN=SEACR_Control)", "SEACR_ControlInTop"
save(reprodD, file = paste0(outPath, "RData/peak_comparisons_macs_narrow_broad_q_0.05_seacr_contro_top10.RData"))

In [ ]:
reprodData = separate(reprodD, "info", into = c("Histone", "CD", "CellType", "Experiment", "HD", "Replicate"), sep = "_") %>% mutate(CellType = factor(toupper(CellType), levels = c("BULK", "N", "CM", "EM", "EMRA")))

reprodData %>% filter(CD == "CD8", CellType != "MAIT", CellType != "NA", Experiment != "PatientIP") %>% ggplot(aes(x = CellType, y = as.numeric(overlapP), fill = CellType)) +
    geom_boxplot() +
    # geom_jitter(aes(color = Experiment), position = position_jitter(0.15)) +
    facet_grid(Histone~caller, scales = "free", space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    ylab("Reproducibility (%)") +
    xlab("") +
    rremove("legend")
ggsave(filename = paste0(fig_path, "cutrun_peak_reproducibility_macs_q_0.05_seacr_top10.pdf"), width = 12, height = 8)

In [ ]:
inPath = "CART_CUTRUN_Project/results/CUTANDRUN/process/"
outPath = "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"
fileList = fread("CART_CUTRUN_Project/meta/Histone_IgG_pairing.csv")$histone

chrom_num <- c()
for(file in fileList){
    print(file)
    macsNarrow = fread(paste0(inPath, file, "/peakCalling/normLibSize/MACS2/macs2_narrow_q_0.05_peaks.narrowPeak")) %>% filter(V1 != "chrY", V1 != "chrM")
    macsBroad = fread(paste0(inPath, file, "/peakCalling/normLibSize/MACS2/macs2_broad_q_0.05_peaks.broadPeak")) %>% filter(V1 != "chrY", V1 != "chrM")
    seacrControl = fread(paste0(inPath, file, "/peakCalling/normLibSize/SEACR/seacr_control_seacrNorm.peaks.stringent.bed")) %>% filter(V1 != "chrY", V1 != "chrM")
    seacrTop = fread(paste0(inPath, file, "/peakCalling/normLibSize/SEACR/seacr_top0.1_spikeInNorm.peaks.stringent.bed")) %>% filter(V1 != "chrY", V1 != "chrM")
    chrom_num <- data.frame(
        chrom_num = c(
            macsNarrow$V1 %>% unique %>% length, 
            macsBroad$V1 %>% unique %>% length,
            seacrControl$V1 %>% unique %>% length,
            seacrTop$V1 %>% unique %>% length
        ),
        type = c("MACS2 Narrow", "MACS2 Broad", "SEACR Control", "SEACR Top"),
        file = file
    ) %>% rbind(chrom_num, .)
}
saveRDS(chrom_num, file = paste0(outPath, "RData/peak_count_chrom_num.rds"))

chrom_num$type = factor(chrom_num$type, levels = c("MACS2 Narrow", "MACS2 Broad", "SEACR Control", "SEACR Top"))
chrom_num %>% ggplot(aes(x = type, y = chrom_num, fill = type)) +
geom_boxplot() +
scale_fill_brewer(palette = "Set1") +
theme_bw(base_size = 20) +
xlab("") +
ylab("Number of Chromosome having Peaks") +
scale_y_continuous(breaks = 1:23) +
rremove("legend") +
rotate_x_text(angle = 90)

ggsave(filename = paste0(fig_path, "cutrun_number_chromosome_with_peak_macs_q_0.05_seacr_top10.pdf"), width = 10, height = 8)

In [ ]:
##	3.5 FriP
# load(file = paste0(outPath, "/RData/masterPeak_peakAnno_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = paste0(outPath, "/RData/IDR_master_peak_list_chromVar_count_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = paste0(outPath, "/RData/countMat_designInfo_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
inMasterPeak = c()
for(hist in hist_list){
  inMasterPeak = colSums(countMat[[hist]]) %>% c(inMasterPeak, .)
}

frPeakTmp = cbind(cbind(frPeak, seqDepth = designInfo$depth), inMasterPeak = inMasterPeak) %>% dplyr::mutate(inPeakProp =  inPeakN/seqDepth * 100, inMasterProp = inMasterPeak/seqDepth * 100)

frPeakD = rbind(frPeakTmp %>% select(-inMasterPeak, -inMasterProp) %>% dplyr::mutate(type = "Individual Peak"), frPeakTmp %>% select(-inPeakN, -inPeakProp) %>% mutate(inPeakN = inMasterPeak, inPeakProp = inMasterProp, type = "Master Peak") %>% select(-inMasterPeak, -inMasterProp))

frPeakD$cell = factor(frPeakD$cell, levels = cell_list)

frPeakD %>% 
  ggplot(aes(x = cell, y = as.numeric(inPeakProp), fill = cell)) +
    geom_boxplot(outlier.shape = NA) +
    geom_jitter(aes(color = expr), position = position_jitter(0.15)) +
    #geom_label_repel(data = frPeakD %>% filter(expr == "Input"), aes(label = paste0(expr, "_", hd)), position = position_jitterdodge()) +
    facet_grid(hist~type, space = "free_x") +
    scale_fill_viridis(discrete = TRUE, option = "viridis", alpha = 0.8) +
    scale_color_viridis(discrete = TRUE, option = "magma") +
    theme_bw(base_size = 20) +
    scale_y_continuous(breaks = c(10, 20, 30, 40, 50, 60, 70, 80, 90, 100)) +
    ylab("FRagment proportion in Peaks regions (FRiPs)") +
    xlab("")
ggsave(filename = paste0(fig_path, "cutrun_FRiPs_macs_q_0.05_seacr_top10.pdf"), width = 12, height = 8)

In [79]:
## DE peaks
# -	Numbers of differentially enriched peak regions (CUT&RUN) in transcription of input of N vs CM vs EM vs EMRA
# o	Venn diagram for H3K27me3 and H3K4me2

## 1. pairwise differential peaks

load(file = paste0(outPath, "/RData/masterPeak_peakAnno_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = paste0(outPath, "/RData/IDR_master_peak_list_chromVar_count_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = paste0(outPath, "/RData/countMat_designInfo_histList_hd1-7_SEACRcontrolTop10_noChrXYM.RData"))
load(file = paste0(outPath, "/RData/results_histList.RData"))

cellList <- c("N", "CM", "EM", "EMRA")
hdList <- paste0("HD", 1:7)
exprList = c("Input", "Product", "Stim1", "Stim2", "Stim3")
cellCompList = c("N_CM", "N_EM", "N_EMRA", "CM_EM", "CM_EMRA", "EM_EMRA")
histList <- c("H3K27me3", "H3K4me2")

inPath <- "CART_CUTRUN_Project/results/CUTANDRUN/process/"
outPath <- "CART_CUTRUN_Project/results/CUTANDRUN/analysis/"
bamDir <- "CART_CUTRUN_Project/results/CUTANDRUN/process/"


deNum = c()
dePeak = vector("list", length(histList))
dePeakIndex = vector("list", length(histList))
for(hist in histList){
  dePeak[[hist]] = list()
  dePeakIndex[[hist]] = list()

  for(exprIter in c("Input", "Product", "Stim1", "Stim2", "Stim3")){
    dePeak[[hist]][[exprIter]] = c()
    for(cellComp in c("N_CM", "N_EM", "N_EMRA", "CM_EM", "CM_EMRA", "EM_EMRA")){
      resData = results[[hist]][[paste(exprIter, cellComp, sep = "_")]]
      deNum = rbind(data.frame(deNum = nrow(resData[Significance != 'notDE']),
                               expr = exprIter,
                               cellComp = cellComp,
                               hist = hist), deNum)
      dePeak[[hist]][[exprIter]] = c(dePeak[[hist]][[exprIter]], resData[Significance != 'notDE']$GeneName)
      dePeakIndex[[hist]][[exprIter]] = c(dePeakIndex[[hist]][[exprIter]], resData[Significance != 'notDE']$GeneIndex)

    }
    dePeak[[hist]][[exprIter]] = dePeak[[hist]][[exprIter]] %>% unique
    dePeakIndex[[hist]][[exprIter]] = dePeakIndex[[hist]][[exprIter]] %>% unique
  }
}

deNum$cellComp = factor(deNum$cellComp, levels = c("N_CM", "N_EM", "N_EMRA", "CM_EM", "CM_EMRA", "EM_EMRA"))

deNum %>% filter(expr == "Input") %>% ggplot(aes(x = cellComp, y = deNum,  fill = cellComp, label = deNum)) +
  geom_bar(stat = "identity") +
  facet_grid(~hist) +
  geom_text(vjust = -0.1) +
  theme_bw(base_size = 20) +
  ggpubr::rotate_x_text(angle = 90) +
  scale_fill_viridis(discrete = TRUE, option = "viridis") +
  scale_color_viridis(discrete = TRUE, option = "magma") +
  ggpubr::rremove("legend") +
  xlab("") +
  ylab("# of Differential Peaks") +
  ggtitle("Input (FDR <= 0.05 |logFC| >=1)")

ggsave(filename = paste0(fig_path, "cutrun_differential_peak_number_macs_q_0.05_seacr_top10.pdf"), width = 12, height = 8)

## 2. compare N vs rest ... for venn diagram

checkRep = function(hist, cell, expr, hd){
  rep = "rep1"
  if(paste(hist, cell, expr, hd, sep = "_") == "H3K27me3_CM_Input_HD1"){
    rep = "rep2"
  }
  if(paste(hist, cell, expr, hd, sep = "_") == "H3K27me3_N_Input_HD2"){
    rep = "rep2"
  }

  return(rep)

}

checkDup = function(expr, hd){

  dupType = "normLibSize" #"norm"
  if(expr == "Input" && hd %in% paste0("HD", 1:3)){
    dupType = "stringent"
  }
  return(dupType)

}

checkBam = function(expr, hd){

  bamType = "bowtie2_align.bam"
  if(expr == "Input" && hd %in% paste0("HD", 1:3)){
    bamType = "bowtie2_align.sorted.rmDup.sortName.bam"
  }
  return(bamType)

}

checkHD = function(expr){

  if(expr == "Input"){
    return(paste0("HD", 4:7))
  }else{
    return(paste0("HD", 1:7))
  }
}
get_contrast <- function(cell_type, design){

  if(cell_type == "N"){
    contrast <- makeContrasts(
      Input_N_Others = Others.Input - N.Input,
      Product_N_Others = Others.Product - N.Product,
      Stim1_N_Others = Others.Stim1 - N.Stim1,
      Stim2_N_Others = Others.Stim2 - N.Stim2,
      Stim3_N_Others = Others.Stim3 - N.Stim3,
      levels = design
    )
  }
  if(cell_type == "CM"){
    contrast <- makeContrasts(
      Input_CM_Others = Others.Input - CM.Input,
      Product_CM_Others = Others.Product - CM.Product,
      Stim1_CM_Others = Others.Stim1 - CM.Stim1,
      Stim2_CM_Others = Others.Stim2 - CM.Stim2,
      Stim3_CM_Others = Others.Stim3 - CM.Stim3,
      levels = design
    )
  }
  if(cell_type == "EM"){
    contrast <- makeContrasts(
      Input_EM_Others = Others.Input - EM.Input,
      Product_EM_Others = Others.Product - EM.Product,
      Stim1_EM_Others = Others.Stim1 - EM.Stim1,
      Stim2_EM_Others = Others.Stim2 - EM.Stim2,
      Stim3_EM_Others = Others.Stim3 - EM.Stim3,
      levels = design
    )
  }
  if(cell_type == "EMRA"){
    contrast <- makeContrasts(
      Input_EMRA_Others = Others.Input - EMRA.Input,
      Product_EMRA_Others = Others.Product - EMRA.Product,
      Stim1_EMRA_Others = Others.Stim1 - EMRA.Stim1,
      Stim2_EMRA_Others = Others.Stim2 - EMRA.Stim2,
      Stim3_EMRA_Others = Others.Stim3 - EMRA.Stim3,
      levels = design
    )
  }
  return(contrast)
}


for(target_cell in c("CM", "EM", "EMRA")){ ## "N"
  target = c()
  for(cell in cellList){
    for(expr in exprList){
      hdL = checkHD(expr)
      for(hd in hdL){
        if( !((cell == "EMRA") & (expr == "Input") && (hd == "HD2"))){
          target = rbind(target, data.frame(cell = cell, expr = expr, humanDonor = hd))
        }

      }
    }
  }
  target$cell[which(target$cell != target_cell)] <- "Others"


  ## Consider human donor as random variable.
  ## Experimental design
  treat <- factor(paste(target$cell, target$expr, sep="."))
  design <- model.matrix(~0 + treat)
  colnames(design) <- levels(treat)
  contrast <- get_contrast(target_cell, design)
  contrast


  results = vector("list", length(histList))
  voomDDS = vector("list", length(histList))
  for(hist in histList){
    ## Filter and delete low expressed genes
      selectR <- which(rowSums(countMat[[hist]]) > 10) ## remove low count genes
      dataS <- countMat[[hist]][selectR,]
      geneNameList = rownames(data)[selectR]

      voomDDS[[hist]] <- voom(counts = dataS, design = design, normalize.method = "cyclicloess", plot = FALSE)
      ## option 1 using voomDDS option2 using normDDS as normalized input.
      inputDDS <- voomDDS[[hist]]
      corfit <- duplicateCorrelation(inputDDS, design, block = target$humanDonor)
      ## corfit$consensus
      fit <- lmFit(inputDDS, design, block = target$humanDonor, correlation = corfit$consensus)
      fitContrast <- contrasts.fit(fit, contrast)
      fitBayes <- eBayes(fitContrast, robust = TRUE)


      results[[hist]] = list()
      for(i in 1:ncol(contrast)){
          ## Results
          res <- topTable(fit = fitBayes, adjust.method = 'fdr', coef = i, number = nrow(inputDDS), sort = 'P')

          res <- data.table(GeneName = geneNameList[as.numeric(rownames(res))], GeneIndex = rownames(res) %>% as.numeric, res)
          res[, Significance := ifelse((adj.P.Val <= 0.05 & sign(logFC) == 1 & abs(logFC) >= 1), 'Up',
                                ifelse((adj.P.Val <= 0.05 & sign(logFC) == -1 & abs(logFC) >= 1), 'Down', 'notDE'))]
          results[[hist]][[i]] = res

          ## Output
          write.table(res, file = paste0(outPath, '/CUTRUN_limma_tables/DE_', colnames(contrast)[i], '_HD1-7_adj0.05_logFC1_one_vs_rest_', target_cell, '.csv'), quote = FALSE, row.names = FALSE, sep = ",")
      }
      names(results[[hist]]) <- colnames(contrast)


  }

  save(results, voomDDS, file = paste0(outPath, "/RData/results_histList_hd1-7_adjustPeak_noChrXYM_one_vs_rest_", target_cell, ".RData"))
} 


In [82]:
hist <- "H3K4me2" ## "H3K4me2"
de_list <- list()
for(target_cell in c("N", "CM", "EM", "EMRA")){
    load(file = paste0(outPath, "/RData/results_histList_hd1-7_adjustPeak_noChrXYM_one_vs_rest_", target_cell, ".RData"))
    de_list[[target_cell]] <- results[[hist]][[paste0("Input_", target_cell, "_Others")]] %>% filter(Significance != "notDE") %$% GeneName
}
# results
pdf(paste0(fig_path, "cutrun_DP_Input_", hist, "_Venn_diagram.pdf"))
venn.diagram(
    x = list(de_list[["N"]], de_list[["CM"]], de_list[["EM"]], de_list[["EMRA"]]),
    category.names = c(
        paste0("N (", length(de_list[["N"]]), ")"), 
        paste0("CM (", length(de_list[["CM"]]), ")"),
        paste0("EM (", length(de_list[["EM"]]), ")"),
        paste0("EMRA (", length(de_list[["EMRA"]]), ")")),
    filename = NULL, #paste0(fig_path, "RNAseq_DE_Input_Venn_diagram.pdf"
    col = c("#440154", "#31688e", "#35b779", "#fde725"), 
    fill = c(alpha("#440154", 0.3), alpha("#31688e", 0.3), alpha("#35b779", 0.3), alpha("#fde725", 0.3)),
    resolution = 1000, 
    disable.logging = TRUE
) %>% grid.draw


dev.off()

png 
  2

In [ ]:
# -	Gene set enrichment analysis of differentially expressed genes  and differentially enriched genes of N vs CM vs EM vs EMRA
# o	Full list of gene set differences

## 1. pairwise DE
load(file = paste0(out_path, "/RData/HD1-3_5-7_normLimma_perExprCondition.RData"))
names(results)
geneID <- fread(paste0(in_path, "RNA_CD8_N_Input_HD1.genes.results"))$gene_id

norm_rna <- voomDDS$E
rownames(norm_rna) <- geneID[selectR]

de_gene_list <- c(
  results$Input_N_CM %>% filter(Significance != "notDE") %$% GeneName,
  results$Input_N_EM %>% filter(Significance != "notDE") %$% GeneName,
  results$Input_N_EMRA %>% filter(Significance != "notDE") %$% GeneName,
  results$Input_CM_EM %>% filter(Significance != "notDE") %$% GeneName,
  results$Input_CM_EMRA %>% filter(Significance != "notDE") %$% GeneName,
  results$Input_EM_EMRA %>% filter(Significance != "notDE") %$% GeneName
) %>% unique
sample_select <- c("Input_N_HD1", "Input_N_HD2", "Input_N_HD3", "Input_N_HD5", 
"Input_N_HD6", "Input_N_HD7", "Input_CM_HD1", 
"Input_CM_HD2", "Input_CM_HD3", "Input_CM_HD5", "Input_CM_HD6", 
"Input_CM_HD7", "Input_EM_HD1", "Input_EM_HD2", "Input_EM_HD3", 
"Input_EM_HD5", "Input_EM_HD6", "Input_EM_HD7", "Input_EMRA_HD1", 
"Input_EMRA_HD2", "Input_EMRA_HD3", "Input_EMRA_HD5", "Input_EMRA_HD6", 
"Input_EMRA_HD7")
norm_rna[de_gene_list, ] %>% dim
norm_rna[de_gene_list, sample_select] %>% head

annotation_col <- data.frame(CellType = rep(c("N", "CM", "EM", "EMRA"), each = 6))
rownames(annotation_col) <- sample_select
annotation_color <- list(CellType = c("N" = "#440154", "CM" = "#31688e", "EM" = "#35b779", "EMRA" = "#fde725"))


test = pheatmap(norm_rna[de_gene_list, sample_select], cluster_rows = TRUE, cluster_cols = FALSE, show_rownames = FALSE, scale = "none", annotation_col = annotation_col, color = PurpleAndYellow(),
annotation_colors = annotation_color, filename = paste0(fig_path, "RNAseq_DE_genes_pairwise_comparison_heatmap.pdf"), width = 7, height = 7)

write.table(norm_rna[de_gene_list[test$tree_row$order], sample_select], file = paste0(fig_path, "RNAseq_DE_genes_pairwise_comparison_heatmap.csv"), quote = F, row.names = TRUE, col.names = TRUE, sep = ",")

## 2. One vs rest
de_gene_list <- c()
for(target_cell in c("N", "CM", "EM", "EMRA")){
    load(file = paste0(out_path, "/RData/HD1-3_5-7_normLimma_perExprCondition_one_vs_rest_", target_cell, ".RData"))
    de_gene_list <- results[[paste0("Input_", target_cell, "_Others")]] %>% filter(Significance != "notDE") %$% GeneName %>% c(de_gene_list, .)
}
de_gene_list <- de_gene_list %>% unique

sample_select <- c("Input_N_HD1", "Input_N_HD2", "Input_N_HD3", "Input_N_HD5", 
"Input_N_HD6", "Input_N_HD7", "Input_CM_HD1", 
"Input_CM_HD2", "Input_CM_HD3", "Input_CM_HD5", "Input_CM_HD6", 
"Input_CM_HD7", "Input_EM_HD1", "Input_EM_HD2", "Input_EM_HD3", 
"Input_EM_HD5", "Input_EM_HD6", "Input_EM_HD7", "Input_EMRA_HD1", 
"Input_EMRA_HD2", "Input_EMRA_HD3", "Input_EMRA_HD5", "Input_EMRA_HD6", 
"Input_EMRA_HD7")
norm_rna[de_gene_list, ] %>% dim
norm_rna[de_gene_list, sample_select] %>% head

annotation_col <- data.frame(CellType = rep(c("N", "CM", "EM", "EMRA"), each = 6))
rownames(annotation_col) <- sample_select
annotation_color <- list(CellType = c("N" = "#440154", "CM" = "#31688e", "EM" = "#35b779", "EMRA" = "#fde725"))
test = pheatmap(norm_rna[de_gene_list, sample_select], cluster_rows = TRUE, cluster_cols = FALSE, show_rownames = FALSE, scale = "none", annotation_col = annotation_col, color = PurpleAndYellow(),
annotation_colors = annotation_color, filename = paste0(fig_path, "RNAseq_DE_genes_one_vs_rest_comparison_heatmap.pdf"), width = 7, height = 7)

write.table(norm_rna[de_gene_list[test$tree_row$order], sample_select], file = paste0(fig_path, "RNAseq_DE_genes_one_vs_rest_comparison_heatmap.csv"), quote = F, row.names = TRUE, col.names = TRUE, sep = ",")



In [ ]:
# GSEA
## 1. Pairwise DE
load(file = paste0(out_path, "/RData/HD1-3_5-7_normLimma_perExprCondition.RData"))
pathway_list <- c("msigdb.v7.4.symbols.gmt", "h.all.v7.4.symbols.gmt", "c2.cp.v7.4.symbols.gmt", "c3.tft.v7.4.symbols.gmt", "c5.all.v7.4.entrez.gmt", "c7.all.v7.4.symbols.gmt")
pathway_name_list <- c("All_Gene_Sets", "Hallmark_All", "C2_noCGP", "C3_TFTonly", "C5_All", "C7_All")
cell_pair_list <- c("N_CM", "N_EM", "N_EMRA", "CM_EM", "CM_EMRA", "EM_EMRA")

for(cell_pair in cell_pair_list){
    print(cell_pair)
    geneList <- results[[paste0("Input_", cell_pair)]]$logFC
    names(geneList) <- results[[paste0("Input_", cell_pair)]]$GeneName %>% gsub(".*_", "", .)
    geneList <- sort(geneList, decreasing = TRUE)

    for(p in 1:length(pathway_list)){
        pathway <- pathway_list[p]
        pathway_name <- pathway_name_list[p]

        module_file <- paste0("/fh/fast/gottardo_r/yezheng_working/SupplementaryData/GSEA/", pathway)
        gene_set <- getGmt(module_file)
        gene_pathway <- geneIds(gene_set)

        gsea_res <- fgsea(pathways = gene_pathway, stats = geneList, minSize = 10, maxSize = 500, eps = 0)
        gsea_res <- gsea_res %>% arrange(padj, decreaing = FALSE)
        fwrite(gsea_res, file = paste0(fig_path, "/RNAseq_DE_gene_pairwise_", cell_pair, "_", pathway_name, "_GSEA.csv"), sep = ",")

        pdf(paste0(fig_path, "/RNAseq_DE_gene_pairwise_", cell_pair, "_", pathway_name, "_GSEA_topEnrichmentScore.pdf"), width = 10, height = 10)
        topPathwaysUp <- gsea_res[ES > 0][head(order(pval), n=10), pathway]
        topPathwaysDown <- gsea_res[ES < 0][head(order(pval), n=10), pathway]
        topPathways <- c(topPathwaysUp, rev(topPathwaysDown)) %>% unique
        plotGseaTable(
            gene_pathway[topPathways], 
            geneList, 
            gsea_res,
            gseaParam=0.5
        )
        dev.off()

        pdf(paste0(fig_path, "/RNAseq_DE_gene_pairwise_", cell_pair, "_", pathway_name, "_GSEA_topPadj.pdf"), width = 10, height = 10)
        collapsedPathways <- collapsePathways(gsea_res[order(padj)][padj < 0.01], 
                                            gene_pathway, geneList)
        mainPathways <- gsea_res[pathway %in% collapsedPathways$mainPathways][
                                order(-NES), pathway]
        plotGseaTable(
            gene_pathway[mainPathways], 
            geneList, 
            gsea_res, 
            gseaParam = 0.5
        )
        dev.off()

    }
}



In [ ]:
## 2. one vs rest DE
pathway_list <- c("msigdb.v7.4.symbols.gmt", "h.all.v7.4.symbols.gmt", "c2.cp.v7.4.symbols.gmt", "c3.tft.v7.4.symbols.gmt", "c5.all.v7.4.entrez.gmt", "c7.all.v7.4.symbols.gmt")
pathway_name_list <- c("All_Gene_Sets", "Hallmark_All", "C2_noCGP", "C3_TFTonly", "C5_All", "C7_All")
target_cell_list <- c("N", "CM", "EM", "EMRA")

for(target_cell in target_cell_list){
    load(file = paste0(out_path, "/RData/HD1-3_5-7_normLimma_perExprCondition_one_vs_rest_", target_cell, ".RData"))
    
    geneList <- results[[paste0("Input_", target_cell, "_Others")]]$logFC
    names(geneList) <- results[[paste0("Input_", target_cell, "_Others")]]$GeneName %>% gsub(".*_", "", .)
    geneList <- sort(geneList, decreasing = TRUE)

    for(p in 1:length(pathway_list)){
        pathway <- pathway_list[p]
        pathway_name <- pathway_name_list[p]

        module_file <- paste0("/fh/fast/gottardo_r/yezheng_working/SupplementaryData/GSEA/", pathway)
        gene_set <- getGmt(module_file)
        gene_pathway <- geneIds(gene_set)

        gsea_res <- fgsea(pathways = gene_pathway, stats = geneList, minSize = 10, maxSize = 500, eps = 0)
        gsea_res <- gsea_res %>% arrange(padj, decreaing = FALSE)
        fwrite(gsea_res, file = paste0(fig_path, "/RNAseq_DE_gene_one_vs_rest_", target_cell, "_", pathway_name, "_GSEA.csv"), sep = ",")

        pdf(paste0(fig_path, "/RNAseq_DE_gene_one_vs_rest_", target_cell, "_", pathway_name, "_GSEA_topEnrichmentScore.pdf"), width = 10, height = 10)
        topPathwaysUp <- gsea_res[ES > 0][head(order(pval), n=10), pathway]
        topPathwaysDown <- gsea_res[ES < 0][head(order(pval), n=10), pathway]
        topPathways <- c(topPathwaysUp, rev(topPathwaysDown)) %>% unique
        plotGseaTable(
            gene_pathway[topPathways], 
            geneList, 
            gsea_res,
            gseaParam=0.5
        )
        dev.off()

    }
}